# Final Project: Honeycomb AKLT as PEPS

Submitted by Yoav Zack, ID 211677398

## `julia` Implementation

We will implement those two wave functions as PEPS in `julia` using `TensorOperations`:

In [ ]:
using LinearAlgebra, TensorOperations, Plots, Printf, BenchmarkTools

First, we define the Honeycomb structure, and some useful functions on it:

In [ ]:
struct HoneycombLattice
    shape::String
    a::Real
    verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}
    bonds::Vector{Tuple}
    ψ#::Vector{Tuple{Integer, Array}}

    function HoneycombLattice(shape::String, a::Real, shift::Tuple{Float64, Float64}, n::Integer, bc::String)
        @assert shape ∈ ("pyramid", "flower", "bricks") "Invalid lattice shape"
        @assert a > 0 "Bond length `a` must be positive"
        @assert length(shift) == 2 "Lattice shift must be of length 2"
        @assert n > 0 "Lattice order `n` must be positive"
        @assert bc ∈ ("periodic", "open") "Invalid lattice boundary condition"

        # define verts location based on lattice shape
        if shape == "pyramid"
            dx = a * sin(π/6)
            dy = a * cos(π/6)
            
            locs = [(0.0, 0.0)]
            for (i,x) in enumerate(range(start=dx, length=n, step=a+dx))
                for y in range(-dy*i, stop=dy*i, length=i+1)
                    append!(locs, [(-x,y), (-x-a,y)])
                end
            end
        
            for (j, y) in enumerate(range(start=-dy*(n-1), stop=dy*(n-1), length=n))
                append!(locs, [(-n*(dx+a)-dx, y)])
            end
        elseif shape == "bricks"
            if n == 1
                locs = [(0.0, 0.0), (0.0, a), (2*a, 0.0), (a, a), (a, 0.0), (2*a, a)]
            else
                locs = [(0.0, 0.0), (0.0, a), (a, 2*a), (a, a)]
                for i in 1:n
                    if i%2==0
                        append!(locs, [r.+(i-1).*(a,0) for r in [(a,2*a),(2*a,2*a),(2*a,a)]])
                    else
                        append!(locs, [r.+(i-1).*(a,0) for r in [(a,0),(2*a,0),(2*a,a)]])
                    end
                end
            end
            
        elseif shape == "flower"
            
        end
            
        # add shift to vertex locations and define verts array
        verts = [(ind, loc .+ shift) for (ind, loc) in enumerate(locs)]
        
        # detect bonds
        bonds = detectbonds(verts, a)

        # remove unwanted bonds in certain geometries
        if shape == "bricks"
            for (ind,bond) in enumerate(bonds)
                if bond[2] == bond[1] + 1 && bond[1] % 3 == 1 && ind != 1
                    deleteat!(bonds, ind)
                end
            end
        end
        
        # create structure
        new(shape, a, verts, bonds, 0)
    end
end

function detectbonds(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, a::Real)
    bonds = []
    for vert in verts
        (ind, loc) = vert
        for (ind_i, loc_i) in nearestNeighboors(verts[ind:end], vert, a) #
            push!(bonds, (ind, ind_i))
        end
    end
    return bonds
end

function nearestNeighboors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, a::Real)
    neighbors = []
    (ind, loc) = vert
    for (ind_i, loc_i) in verts
        d = sum((loc .- loc_i).^2)
        if abs(d - a) < 1e-1*a
            push!(neighbors, (ind_i, loc_i))
        end
    end
    return neighbors
end

function showlattice(fig::Plots.Plot, lat::HoneycombLattice, dolabel::Bool)
    x = [loc[1] for (ind, loc) in lat.verts]
    y = [loc[2] for (ind, loc) in lat.verts]
    for b in lat.bonds
        dx = [lat.verts[i][2][1] for i in b]
        dy = [lat.verts[i][2][2] for i in b]
        plot!(fig, dx, dy, label="", color="blue")
    end
    scatter!(fig, x, y, aspect_ratio=1, label="", color="red")

    if dolabel
        for (ind, loc) in lat.verts
            annotate!(fig, loc[1], loc[2], ind)
        end
    end
    return fig
end

In [ ]:
lat = HoneycombLattice("bricks", 1, (5.0,1.0), 1, "open")
fig = scatter()
fig = showlattice(fig, lat, true)

In [ ]:
lat = HoneycombLattice("pyramid", 1, (5.0,1.0), 6, "open")
fig = scatter()
fig = showlattice(fig, lat, false)